In [1]:
import os
import sys

# 예제 문서 
# PDF 문서 로드 및 텍스트 추출
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS

from langchain_openai import OpenAIEmbeddings


# utils 폴더의 경로를 추가
sys.path.append(os.path.abspath(os.path.join('..', 'utils')))

# sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('..'))))
from config import config

In [3]:
## DB 생성 


DBfolder = "../dataDB/PDFvectorDB"  

embeddings = OpenAIEmbeddings( model=config['embed_model']['model_name'] )

database = Chroma(
                persist_directory=DBfolder,         # 이미 DB가 있으면 불러오고 없으면 새로 만듬
                embedding_function = embeddings
)

# database = FAISS.from_documents(
#                             docs,
#                             persist_directory=folder 
#                             embeddings)
# )

# # 리스트 내용을 각각 document로 변환
# documents = [Document(page_content=text_list[i], metadata={'category':meta[i]}) for i in range(data.shape[0])]

# # Insert
# database.add_documents(documents)

### 새롭게 입력된 PDF 

In [14]:
# 예제 문서 
# PDF 문서 로드 및 텍스트 추출
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyMuPDFLoader

file_path = "../../raw_data/[법률_규범_특허][코트라][2022]한-인도 CEPA 활용법 및 인도 통상 애로.pdf"

# loader = PyPDFLoader(file_path, extract_images=False)
loader = PyMuPDFLoader(file_path, extract_images=False)

docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

from langchain_text_splitters import RecursiveCharacterTextSplitter

# 단계 2: 문서 분할(Split Documents)
# HugoingFace Embedding 모델의 Tokenizer를 사용하여 토큰화
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# text_splitter = RecursiveCharacterTextSplitter(tokenizer=tokenizer, chunk_size=1000, chunk_overlap=50)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)


split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

문서의 페이지수: 41
분할된 청크의수: 58


In [19]:
docs[:10]

[Document(page_content='사례로 정리해보는\n한-인도 CEPA \n활용법 및 인도 통상 애로\nKOTRA자료 22-055\n', metadata={'source': '../../raw_data/[법률_규범_특허][코트라][2022]한-인도 CEPA 활용법 및 인도 통상 애로.pdf', 'file_path': '../../raw_data/[법률_규범_특허][코트라][2022]한-인도 CEPA 활용법 및 인도 통상 애로.pdf', 'page': 0, 'total_pages': 41, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign 17.2 (Macintosh)', 'producer': 'Adobe PDF Library 16.0.7', 'creationDate': "D:20220610145706+09'00'", 'modDate': "D:20220614154828+09'00'", 'trapped': ''}),
 Document(page_content='인사말\n한-인도 CEPA 발효 11년이 지난 2021년, 양국 간 교역액은 약 236억 달러를 달성하며 역대 최고치\n를 기록하였습니다. 한국의 대인도 수출과 수입 모두 전년 대비 약 30%, 64% 상승하며, 무역수지는 75\n억 달러 흑자를 기록하였습니다. 인도는 한국의 대인도 수출품 85%에 대한 관세를 철폐 또는 감축하면\n서, 관세장벽은 많이 허물어졌습니다.\n그러나 인도는 한국산 물품에 대해 원산지관리를 강화하고, 세관 담당자가 임의적으로 품목분류를 변\n경하여 관세 추징을 하는 등 한국기업이 CEPA 특혜관세를 적용 받기 어렵게 만들고 있습니다. 아울러, \n인도는 BIS 인증 대상을 확대하고 수입모니터링 시스템을 시행하며 비관세장벽을 높이고 있습니다.\nKOTRA는 이러한 환경에서 우리 기업을 지원하기 위해 2018년 뉴델리 무역관, 2019년 첸

In [17]:
split_documents[:10]

[Document(page_content='사례로 정리해보는\n한-인도 CEPA \n활용법 및 인도 통상 애로\nKOTRA자료 22-055', metadata={'source': '../../raw_data/[법률_규범_특허][코트라][2022]한-인도 CEPA 활용법 및 인도 통상 애로.pdf', 'file_path': '../../raw_data/[법률_규범_특허][코트라][2022]한-인도 CEPA 활용법 및 인도 통상 애로.pdf', 'page': 0, 'total_pages': 41, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign 17.2 (Macintosh)', 'producer': 'Adobe PDF Library 16.0.7', 'creationDate': "D:20220610145706+09'00'", 'modDate': "D:20220614154828+09'00'", 'trapped': ''}),
 Document(page_content='인사말\n한-인도 CEPA 발효 11년이 지난 2021년, 양국 간 교역액은 약 236억 달러를 달성하며 역대 최고치\n를 기록하였습니다. 한국의 대인도 수출과 수입 모두 전년 대비 약 30%, 64% 상승하며, 무역수지는 75\n억 달러 흑자를 기록하였습니다. 인도는 한국의 대인도 수출품 85%에 대한 관세를 철폐 또는 감축하면\n서, 관세장벽은 많이 허물어졌습니다.\n그러나 인도는 한국산 물품에 대해 원산지관리를 강화하고, 세관 담당자가 임의적으로 품목분류를 변\n경하여 관세 추징을 하는 등 한국기업이 CEPA 특혜관세를 적용 받기 어렵게 만들고 있습니다. 아울러, \n인도는 BIS 인증 대상을 확대하고 수입모니터링 시스템을 시행하며 비관세장벽을 높이고 있습니다.\nKOTRA는 이러한 환경에서 우리 기업을 지원하기 위해 2018년 뉴델리 무역관, 2019년 첸나이

In [20]:
# documents = [Document(page_content=text, metadata={'category':'윤영인 기자단'}) for text in text_list]

database.add_documents(split_documents)

['fd598c10-66f1-4c44-b68b-39d922b12910',
 'f8fdb529-57fc-4b5c-b3ec-2e749980229a',
 '4fe57113-05bb-482a-8493-941d0eb6039a',
 '3b4f72b4-e8b7-44d5-8c93-ffa703a22946',
 'fbeb245d-14b5-47b3-ac04-d0abdfa92f28',
 '37bcf7fd-23ab-46e1-aa27-5ee5655f174d',
 'c82530ae-e92f-4104-9637-822b2aa4724d',
 '34749c8d-4f5e-446d-aee3-122b9d264fb9',
 'b0e0e316-afbb-45f9-a9ab-7604f6605318',
 '124bd102-8460-4d91-b8c5-f5dd57046d06',
 '704c8b6b-78b4-4ad3-a1cc-f7fd5636762d',
 '71a0b7a1-57f8-4a79-904b-9f22beceb3ba',
 '39f335a1-267f-479b-91c2-574564a0eb55',
 '42eb4381-0c8a-4b28-81df-ebbd36e050e8',
 '432b85a9-c924-49d2-9eed-f2053983eb0a',
 '0d35b40a-b44b-4a9a-9e18-b0c3cf9463f4',
 '88132a13-0d9a-4382-b4bb-9c457023c77a',
 '884ca8f3-6d9e-49c7-9ec7-ea5ec655c8d5',
 '780e5edf-9b74-4179-aa82-797b728914a1',
 '2cc92d2c-7ab4-4779-8b8c-657924c2d32f',
 '8b2cda88-b5b4-422b-b7da-f68778bbe61f',
 '9d4fefe8-597a-4e54-a914-3102b7b63988',
 'b613494f-e410-4c64-8be8-f023dd8729af',
 '7362e276-9c8c-42ab-b99e-c3b6bdb4c7aa',
 '8c7abdbf-38a8-